In [1]:
#ENCRYPT AT CLIENT SIDE

In [2]:
import numpy as np
import pandas as pd
from wrapper import SEAL, t_list
import ctypes

In [3]:
def extended_Euclidean_algorithm(a, b):
	b0 = b
	x0, x1 = 0, 1
	if b == 1: return 1
	while a > 1:
		q = a // b
		a, b = b, a%b
		x0, x1 = x1 - q * x0, x0
	if x1 < 0: x1 += b0
	return x1

def chinese_remainder_theorem(array):
	result = 0
	for t_index in range(len(array)):
		result += array[t_index].item() * bezout_coefficients[t_index] * t_product_over_t[t_index]
	return result % t_product

def crt_inverse(tensor):
	examples_count = tensor.shape[0]
	temp = np.empty(tensor.shape[:-1], dtype=object)
	for i in range(examples_count):
		for j in range(2):
			temp[i, j] = chinese_remainder_theorem(tensor[i, j, :])
			if (temp[i, j]>negative_threshold):
				temp[i, j] = temp[i, j] - t_product
	return temp

In [4]:
# CRT PARAMETERS
# compute the producte of all t, and the threshold for negative numbers:
#   t_product
#   negative_threshold
t_product = 1
for t_index in range(len(t_list)):
	t_product = t_product * t_list[t_index]
negative_threshold = t_product // 2
# compute t_product // t and the Bezout coefficients, for all t: 
#   t_product_over_t
#   bezout_coefficients
t_product_over_t = []
bezout_coefficients = []
for t_index in range(len(t_list)):
	t_product_over_t.append(t_product // t_list[t_index])
	temp = extended_Euclidean_algorithm(t_product_over_t[t_index], t_list[t_index])
	bezout_coefficients.append(temp)

In [5]:
class Client:
    
    def __init__(self, user_id):
        self.user_id = user_id        
        self.SEALobj = SEAL()        
        self.t_list = self.SEALobj.t_list
        self.t_size = len(self.SEALobj.t_list)
        self.precision = 10
        
        
        self.sample = None
        self.encrypted_sample = None
        self.encrypted_result = None
        self.result = None
        
                        
    def generate_keys(self):
        self.lib = ctypes.cdll.LoadLibrary('./SEAL/libseal.so')
        for i in range(5):
            if (not (os.path.isfile("./keys/evaluation-"+str(i))
                    and os.path.isfile("./keys/public-"+str(i))
                    and os.path.isfile("./keys/secret-"+str(i))
                    )):
                print("Key missing: generating new keys...")
                self.lib.generate_new_keys()
                break
                
    """
    Raw sample data:
    input_data = np.array(shape = (1, 30), dtype='f')
    """
    def receive_sample(self, input_data):
        self.sample = input_data
    
    def receive_encrypted_result(self, data):
        self.encrypted_result = data
        
        
    """Encode: converting float numbers n to int((n * precision) % t_i)
    Input: input_data type(np.float32), shape=(1,30)
    Output: encoded_sample type(np.uint64), shape=(1,30,5)
    """
    def encode_sample(self, input_data, precision=10):
        encoded_input = np.empty(shape=(input_data.shape[0], input_data.shape[1], self.t_size), dtype=np.uint64)
        for i in range(input_data.shape[0]): #1
            for j in range(input_data.shape[1]): #30
                value = round(input_data[i,j].item()*precision)
                for t in range(self.t_size):
                    encoded_input[i, j, t] = value % self.t_list[t]
        return encoded_input
    
    

    def encrypt_sample(self, encoded_sample):
        return self.SEALobj.encrypt_tensor(encoded_sample)
    
    def decrypt_sample(self, encrypted_output):
        return self.SEALobj.decrypt_tensor(encrypted_output,1)
    
        
    def get_encrypted_sample(self):
        return self.encrypted_sample
    
    def get_SEAL_obj(self):
        return self.SEALobj
    

In [6]:
client = Client(1)
# a = np.array([[ 0.49313486, -0.34081736,  0.50191972,  0.38008119,  0.05704016,
#         0.46813485,  0.16016434,  0.45979075,  0.12956228, -0.28383312,
#         0.34441204, -0.41194671,  0.3192262 ,  0.2863989 , -0.42557394,
#         0.85224107, -0.12259865,  0.14692686, -0.06850089,  0.17728197,
#         0.88944082,  0.26251982,  0.89340995,  0.76063247,  0.35957399,
#         1.9153767 ,  0.62527495,  1.02842109,  1.39647083,  1.17874205]], dtype='f')
#sample = 1
a = np.array([[-0.25457009,  0.33425833, -0.30580282, -0.31300905, -1.61101217,
       -0.96094791, -0.8747604 , -0.92899824, -0.36516108, -0.57218419,
       -0.2942574 ,  0.49787915, -0.2691093 , -0.30335667,  0.19926576,
       -0.28011719, -0.72351751, -1.04820266,  0.02059392, -0.44144905,
       -0.34781631,  0.52827207, -0.3385181 , -0.39293154, -0.89173759,
       -0.55317418, -0.92232003, -1.08716651, -0.33034   , -0.53662011]], dtype='f')
client.receive_sample(a)

In [7]:
client.sample

array([[-0.2545701 ,  0.33425832, -0.30580282, -0.31300905, -1.6110122 ,
        -0.96094793, -0.8747604 , -0.92899823, -0.3651611 , -0.5721842 ,
        -0.2942574 ,  0.49787915, -0.2691093 , -0.30335668,  0.19926576,
        -0.28011718, -0.72351754, -1.0482026 ,  0.02059392, -0.44144905,
        -0.34781632,  0.5282721 , -0.3385181 , -0.39293155, -0.8917376 ,
        -0.5531742 , -0.92232   , -1.0871665 , -0.33034   , -0.5366201 ]],
      dtype=float32)

In [8]:
encoded = client.encode_sample(client.sample)

In [9]:
encrypted =client.encrypt_sample(encoded)

In [10]:
SealObj = client.get_SEAL_obj()

In [11]:
# np.save("./encrypted_input", c)

# SERVER

In [12]:
#COMPUTE AT SERVER SIDE

In [13]:
import numpy as np
from wrapper import SEAL

In [14]:
def to_object_dtype(tensoreuint):
        shape = tensoreuint.shape
        tensoreuint.shape = (tensoreuint.size,)
        new = np.empty((tensoreuint.size,), dtype=object)
        for i in range(tensoreuint.size):
            new[i] = int(tensoreuint[i].item())
        new.shape = shape
        return new

In [15]:
class NeuralNetwork():
    def __init__(self):      
        self.dense1_kernel = np.load("./nn_data/dense1_kernel.npy")
        self.dense1_bias = np.load("./nn_data/dense1_bias.npy")
        self.dense2_kernel = np.load("./nn_data/dense2_kernel.npy")
        self.dense2_bias = np.load("./nn_data/dense2_bias.npy")
        self.dense3_kernel = np.load("./nn_data/dense3_kernel.npy")
        self.dense3_bias = np.load("./nn_data/dense3_bias.npy")
    
    
        self.SEALobj = None
        
    def set_SEAL_obj(self, SEALobj):
        if(self.SEALobj == None):
            self.SEALobj = SEALobj
            self.q_list = self.SEALobj.q_list
            self.k_list = self.SEALobj.k_list
            self.n_parm = self.SEALobj.n_parm
            self.enc_poly_size = self.SEALobj.enc_poly_size
            self.q_size = len(self.q_list)
            self.t_size = len(self.SEALobj.t_list)
    """input_data = np.array(shape=(1,30,5))
    """
#     def get_encrypted_input(self, input_data):
#         self.encrypted_input = input_data
        
    def predict(self, SEALobj, encrypted_input):
        #If SEALobj == None then set SEALobj to Server
        self.set_SEAL_obj(SEALobj)
        
        encrypted_output = np.empty((encrypted_input.shape[0],30,self.t_size), dtype=np.uint64)
        poly_groups_count = encrypted_input.shape[0]//self.enc_poly_size
        
        #LEYER 1: Fully connected layer
        encrypted_output = to_object_dtype(encrypted_input)
        dense1_kernel = to_object_dtype(self.dense1_kernel)
        
        ## kernel
        temp = np.empty((encrypted_output.shape[0],30,self.t_size), dtype=object)
        for t_index in range(self.t_size):
            temp[...,t_index] = encrypted_output[...,t_index].dot(dense1_kernel[...,t_index])
        encrypted_output = temp
        temp = None
        ## % q
        for axis1 in range(encrypted_output.shape[1]):
            for axis2 in range(encrypted_output.shape[2]):
                for poly_group_index in range(poly_groups_count):
                    for size_index in range(2):
                        for q_index in range(self.q_size):
                            for n_index in range(self.n_parm+1):
                                axis0 = poly_group_index*self.enc_poly_size + size_index*self.q_size*(self.n_parm+1) + q_index*(self.n_parm+1) + n_index
                                temp = encrypted_output[axis0,axis1,axis2]
                                temp = temp % self.q_list[q_index]
                                encrypted_output[axis0,axis1,axis2] = temp
        ## bias
        for axis1 in range(encrypted_output.shape[1]):
            for axis2 in range(encrypted_output.shape[2]):
                for poly_group_index in range(poly_groups_count):
                    for q_index in range(self.q_size):
                        axis0 = poly_group_index*self.enc_poly_size + ((self.n_parm+1)*q_index)
                        temp = encrypted_output[axis0,axis1,axis2]
                        temp = temp + self.dense1_bias[axis1,axis2].item()*self.k_list[axis2][q_index]
                        temp = temp % self.q_list[q_index]
                        encrypted_output[axis0,axis1,axis2] = temp
                        
        
        #LAYER2: square activation function
        encrypted_output = self.SEALobj.square_tensor(encrypted_output)
        
        #LAYER3: fully connected layer
        encrypted_output = to_object_dtype(encrypted_output)
        dense2_kernel = to_object_dtype(self.dense2_kernel)
        
        ## kernel
        temp = np.empty((encrypted_output.shape[0],16,self.t_size), dtype=object)
        for t_index in range(self.t_size):
            temp[...,t_index] = encrypted_output[...,t_index].dot(dense2_kernel[...,t_index])
        encrypted_output = temp
        temp = None
        ## % q
        for axis1 in range(encrypted_output.shape[1]):
            for axis2 in range(encrypted_output.shape[2]):
                for poly_group_index in range(poly_groups_count):
                    for size_index in range(2):
                        for q_index in range(self.q_size):
                            for n_index in range(self.n_parm+1):
                                axis0 = poly_group_index*self.enc_poly_size + size_index*self.q_size*(self.n_parm+1) + q_index*(self.n_parm+1) + n_index
                                temp = encrypted_output[axis0,axis1,axis2]
                                temp = temp % self.q_list[q_index]
                                encrypted_output[axis0,axis1,axis2] = temp
        ## bias
        for axis1 in range(encrypted_output.shape[1]):
            for axis2 in range(encrypted_output.shape[2]):
                for poly_group_index in range(poly_groups_count):
                    for q_index in range(self.q_size):
                        axis0 = poly_group_index*self.enc_poly_size + ((self.n_parm+1)*q_index)
                        temp = encrypted_output[axis0,axis1,axis2]
                        temp = temp + self.dense2_bias[axis1,axis2].item()*self.k_list[axis2][q_index]
                        temp = temp % self.q_list[q_index]
                        encrypted_output[axis0,axis1,axis2] = temp
                        
        
        #LAYER4: square activation function
        encrypted_output = self.SEALobj.square_tensor(encrypted_output)

        #LAYER5: fully connected layer
        encrypted_output = to_object_dtype(encrypted_output)
        dense3_kernel = to_object_dtype(self.dense3_kernel)
        
        ## kernel
        temp = np.empty((encrypted_output.shape[0],2,self.t_size), dtype=object)
        for t_index in range(self.t_size):
            temp[...,t_index] = encrypted_output[...,t_index].dot(dense3_kernel[...,t_index])
        encrypted_output = temp
        temp = None
        ## % q
        for axis1 in range(encrypted_output.shape[1]):
            for axis2 in range(encrypted_output.shape[2]):
                for poly_group_index in range(poly_groups_count):
                    for size_index in range(2):
                        for q_index in range(self.q_size):
                            for n_index in range(self.n_parm+1):
                                axis0 = poly_group_index*self.enc_poly_size + size_index*self.q_size*(self.n_parm+1) + q_index*(self.n_parm+1) + n_index
                                temp = encrypted_output[axis0,axis1,axis2]
                                temp = temp % self.q_list[q_index]
                                encrypted_output[axis0,axis1,axis2] = temp
        ## bias
        for axis1 in range(encrypted_output.shape[1]):
            for axis2 in range(encrypted_output.shape[2]):
                for poly_group_index in range(poly_groups_count):
                    for q_index in range(self.q_size):
                        axis0 = poly_group_index*self.enc_poly_size + ((self.n_parm+1)*q_index)
                        temp = encrypted_output[axis0,axis1,axis2]
                        temp = temp + self.dense3_bias[axis1,axis2].item()*self.k_list[axis2][q_index]
                        temp = temp % self.q_list[q_index]
                        encrypted_output[axis0,axis1,axis2] = temp
                        
        return encrypted_output                
        

In [16]:
nn = NeuralNetwork()

In [17]:
output = nn.predict(SealObj, encrypted)

# CLIENT

In [18]:
#DECRYPT AT CLIENT SIDE

In [19]:
decrypted = client.decrypt_sample(output)

In [20]:
predict = crt_inverse(decrypted)

In [21]:
result = np.argmax(predict, axis=1)
result

array([1])